<a href="https://colab.research.google.com/github/stewartjollymore/MNIST_NNs/blob/main/Numpy%20NNs/Fully_Connected_singlepass_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical


#Below we separate and arange the data as seen around the web
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#Creating the layer dimensions

input_size1 = 28*28
output_size1 = 100
input_size2 = 100
output_size2 = 50
input_sizefinal = 50
output_sizefinal = 10

In [42]:
import numpy as np

#Initialze weights and biases

w1 = np.random.rand(input_size1, output_size1) - 0.5
b1 = np.random.rand(1, output_size1) - 0.5

w2 = np.random.rand(input_size2, output_size2) - 0.5
b2 = np.random.rand(1, output_size2) - 0.5

w3 = np.random.rand(input_sizefinal, output_sizefinal) -0.5
b3 = np.random.rand(1, output_sizefinal) - 0.5

In [ ]:
import tensorflow as tf
import keras as ks

#Define activations for forward prop and backprop

def sigmd(x):
  return  1/(1+np.exp(-x))

def gradient(x):
  return sigmd(x)*(1-sigmd(x))


In [43]:
#Single pass for a single data element

single_xtrain = x_train[0]
single_ytrain = y_train[0]

#Pass through first hidden layer and activation neruon
layer1 = np.dot(single_xtrain,w1)+b1
act_layer1 = sigmd(layer1)

#Pass through second hidden layer and activation neuron
layer2 = np.dot(act_layer1,w2)+b2
act_layer2 = sigmd(layer2)

#Pass through output layer and acrivation neuron
layer3 = np.dot(act_layer2,w3)+b3
output = sigmd(layer3)

In [ ]:
#define the error metrics

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_grad(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [44]:
#Mesuring measn squared error
output_error = mse(single_ytrain, output)
output_error

0.3349214564150429

In [45]:
#Backward Propigation steps

#Define batch size and learning rate
N = single_ytrain.size
learning_rate = 0.1

#calculate the output error
#output_error = mse_grad(single_ytrain,output)
output_error = single_ytrain - output

#Backprop through output layer
output_grad = output_error*gradient(output)
w3_grad = np.dot(layer2.T, output_grad)
b3_grad = np.sum(output_grad)
layer2_grad = np.dot(output_grad, w3.T)*gradient(act_layer2)

w2_grad = np.dot(layer1.T, layer2_grad)
b2_grad = np.sum(layer2_grad, axis = 0, keepdims=True)
layer1_grad = np.dot(layer2_grad,w2.T)*gradient(act_layer1)

w1_grad = np.dot(single_xtrain.T, layer1_grad)
b1_grad = np.sum(layer1_grad, axis = 0, keepdims = True)

#Updating the new weights by magnitude of the learning rate
w3_new = learning_rate*w3_grad
w2_new = learning_rate*w2_grad
w1_new = learning_rate*w1_grad

#Updating the new biases
b3_new = learning_rate*b3_grad
b2_new = learning_rate*b2_grad
b1_new = learning_rate*b1_grad

In [46]:
#Second pass through forward direction and
#error calculation

layer1_new = np.dot(single_xtrain,w1_new)+b1_new
act_layer1_new = sigmd(layer1_new)

layer2_new = np.dot(act_layer1_new,w2_new)+b2_new
act_layer2_new = sigmd(layer2_new)

layer3_new = np.dot(act_layer2_new,w3_new)+b3_new
output_new = sigmd(layer3_new)

mse(single_ytrain, output_new)

0.22540716501085561

In [48]:
print(output)
print(output_new)

[[0.67424512 0.24510045 0.90499412 0.72857314 0.65836604 0.64413475
  0.69447591 0.30568622 0.35651924 0.47092244]]
[[0.468139   0.47150177 0.46684466 0.46779721 0.46824303 0.47674066
  0.46800916 0.47096968 0.47053464 0.46960164]]
